# Features Extraction & Clustering. Part III.
This part is focused on clustering with taking market states into account.

In [1]:
# Import of necessary libs and our classes
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial.distance import squareform, pdist
from sklearn.cluster import DBSCAN
from cj_loader import Storer, Extractor, extract_features
import warnings
warnings.filterwarnings("ignore")

# Init storer object with given data and calculate precense of data
storer = Storer()

## Updates in Extractor class
Now it's possible to init new instance of **Extractor** class by passing **periods_of_interest** argument. This will slice the scope of coin by time axis in accordance to periods of interest. This parameter was implemented especially to use slicing with result of **bull_bear_periods()** function execution. Below is the code to extract arrays of stable, bullish and bearish periods of interest. These arrays is exactly what **Exctractor** awaits as **periods_of_interest** arg.

**Important**: cut_start and periods_of_interest arguments are mutually exclusive. It is expected that periods are extracted for whole timeline of main frame.

## Market states
Firstly we have to obtain periods of different market states in accordance with bullish, bearish or stable BTC trends. As previously we will use **bull_bear_periods()** method of **Extractor** class. See **Market State Detection** report for details.

In [2]:
# Extract info for BTC
btc = Extractor(storer.mf['bitcoin'])

# Define parameters to extract BTC bullish, bearish and stable periods. (parameters can be adjusted)
time_window = 10
threshold = 0.1

# Obtain periods of interest
periods = btc.bull_bear_periods(base_t_len_days=time_window, thresh=threshold) 
stable = periods == 0
bullish = periods == 1
bearish = periods == -1

In order to perform clustering with our basic features extractor, firstly we have perform top-level clustering that will separate coins to groups in correspondance to presence of data. See part II of this report for details.

In [3]:
precense = storer.applicability()

# Calculation of data completeness matrix and weights
data_compl = storer.data_completeness()
weights = data_compl.mean().sort_values()

# transform boolean matrix to numeric
weighted = precense.copy()
for index, row in precense.iterrows():
    weighted.loc[index][:] = row * weights

# calculate distances between pairs of coins
distances = pd.DataFrame(squareform(pdist(weighted)), index=weighted.index, columns=weighted.index)

# Top-level clustering with DBSCAN algorithm
clustering = DBSCAN(eps=0.3, min_samples=3).fit(distances)
labels = clustering.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print('Estimated number of clusters: %d' % n_clusters_)

# Clusters aggregation
weighted['label'] = labels
clusters = {}
for label in np.unique(labels):
    cl = weighted[weighted['label'] == label]
    clusters[label] = cl

Estimated number of clusters: 4


This time we have three different market states and corresponding time periods, so we have to create three sets in accordance to states and perform clustering for each of them.

In [4]:
# Extracting features
cl_coin_features_stable = {}
cl_coin_features_bullish = {}
cl_coin_features_bearish = {}
top_level_outliers = set()
for label, cluster in clusters.items():
    if label != -1:
        cl_coin_features_stable[label] = extract_features(storer, periods_of_interest=stable, coins_set=cluster.index)
        cl_coin_features_bullish[label] = extract_features(storer, periods_of_interest=bullish, coins_set=cluster.index)
        cl_coin_features_bearish[label] = extract_features(storer, periods_of_interest=bearish, coins_set=cluster.index)
    else:
        top_level_outliers.update(cluster.index.values.tolist())

We will use HDBSCAN to obtain clusters as in previous part of this report. 

In [5]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [6]:
import hdbscan

# Simple function to print results of clustering for different groups
def print_clusters(cl_coin_features):
    unclustered_coins = set()
    for top_level_cl in range(n_clusters_):
        print("Top-level cluster: {}".format(top_level_cl))
        tc = cl_coin_features[top_level_cl]
        tc = tc.replace([np.inf, -np.inf], np.nan)
        tc = tc.dropna(axis='rows', how='all')
        tc = tc.dropna(axis='columns', how='any')
        print("shape: {}".format(tc.shape))
        if tc.shape[1] > 0 and tc.shape[0] > 2:
            clusterer = hdbscan.HDBSCAN(min_cluster_size=2)
            ll_labels = clusterer.fit_predict(tc)
            tc['label'] = ll_labels
            for label in np.unique(ll_labels):
                if label != -1:
                    print('label %d' %label)
                    cl = tc[tc['label'] == label]
                    display(cl)
                    print('-'*40)
                else:
                    noise_coins = tc[tc['label'] == label].index.values.tolist()
                    unclustered_coins.update(noise_coins)
        else:
            unclustered_coins.update(tc.index.values.tolist())
            
    print('='*40) 
    print("Unclustered coins:")
    for co in unclustered_coins:
        print(co)

## Clustering in bearish periods
Since combination of using algorithm and passed parameters detect one short bearish period (beginning of 2018 after BTC drop), results for this state are rather of an introductory nature. There are plenty methods of fixing it, the right direction is to make **bull_bear_periods()** less "rigorous". Using moving average as kernel possibly can make this done.

In [7]:
print_clusters(cl_coin_features_bearish)

Top-level cluster: 0
shape: (26, 20)
label 0


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,txcount_mean,txcount_median,txcount_stddev,txcount_skewns,txcount_kurtos,...,days_exist_median,days_exist_stddev,days_exist_skewns,days_exist_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
0x,1768.6,1622.5,463.548315,1.736606,2.587618,2092.8,1822.0,732.592083,1.886774,2.545445,...,153.5,2.872281,0.0,-1.224242,1.165574,1.135887,0.094853,1.249310,-0.035413,0
basic-attention-token,1728.5,1734.5,362.563167,-0.212329,-0.925524,1970.2,2076.5,451.041195,-0.074247,0.022512,...,227.5,2.872281,0.0,-1.224242,1.139517,1.129212,0.112197,0.098189,-1.580011,0
icon,1716.7,1746.5,209.763224,-0.183685,-1.427899,1723.7,1826.5,305.879077,-0.281439,-1.305513,...,114.5,2.872281,0.0,-1.224242,0.997230,0.999552,0.065978,-0.307636,-1.123565,0
power-ledger,1503.6,1322.0,414.172959,0.932425,-0.571460,1731.0,1487.5,552.524027,0.891156,-0.651663,...,222.5,2.872281,0.0,-1.224242,1.138981,1.139660,0.053293,0.066490,0.088249,0
status,1656.8,1641.0,302.678972,-0.062111,-0.507579,1909.6,1987.5,358.422432,0.041024,-0.888805,...,206.5,2.872281,0.0,-1.224242,1.153334,1.149394,0.057477,-0.977740,0.553725,0
funfair,2173.8,1960.5,472.240151,0.641201,-0.560855,2488.6,2335.0,607.571428,1.519046,1.919902,...,198.5,2.872281,0.0,-1.224242,1.146148,1.167786,0.102542,-0.122396,-1.292892,0
vechain,2026.7,2021.0,265.957910,0.097076,0.442716,2244.7,2190.0,280.347659,0.603075,-0.058198,...,149.5,2.872281,0.0,-1.224242,1.109696,1.098557,0.041685,0.399670,-0.150095,0


----------------------------------------
label 1


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,txcount_mean,txcount_median,txcount_stddev,txcount_skewns,txcount_kurtos,...,days_exist_median,days_exist_stddev,days_exist_skewns,days_exist_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
zilliqa,103.25,103.25,73.976582,0.000000,-1.360000,102.125,102.125,73.883413,0.000000,-1.360000,...,2.5,1.118034,0.0,-1.360000,0.780937,0.780937,0.023059,-1.450794e-14,-1.360000,1
aion,804.90,616.00,474.750766,1.506789,0.711344,841.200,716.500,472.586458,1.301572,0.394990,...,92.5,2.872281,0.0,-1.224242,1.051131,1.027232,0.109949,4.969964e-01,-0.802425,1
qash,580.40,581.50,57.077491,0.041575,-0.917794,680.500,678.500,77.592848,0.196196,-1.164200,...,93.5,2.872281,0.0,-1.224242,1.171970,1.166819,0.057198,2.434571e-01,-1.001334,1
loopring,791.20,734.50,175.058162,1.022809,0.085609,870.500,764.500,264.653075,1.407778,1.193790,...,163.5,2.872281,0.0,-1.224242,1.085309,1.062730,0.087813,1.027320e+00,0.363327,1
bytom,843.30,837.50,100.804811,0.095573,-1.216481,886.300,866.500,139.405201,0.643502,-0.527229,...,182.5,2.872281,0.0,-1.224242,1.047300,1.033786,0.051368,1.191575e+00,0.594967,1
ethos,679.50,691.50,131.672510,0.171720,-0.650915,660.800,703.000,163.011533,-0.057198,-0.956008,...,198.5,2.872281,0.0,-1.224242,0.962112,0.986129,0.072373,-2.522425e-01,-1.117407,1
kucoin-shares,48.70,47.00,9.757561,0.635097,-0.804168,50.200,43.500,16.061133,0.959327,-0.512521,...,142.5,2.872281,0.0,-1.224242,1.021760,0.942308,0.198682,8.287900e-01,-0.513364,1
aelf,399.80,389.00,66.642029,1.305741,1.410491,524.200,496.000,145.198347,1.632919,2.322106,...,24.5,2.872281,0.0,-1.224242,1.293221,1.268053,0.142003,9.040301e-01,0.009261,1
kyber-network,899.20,777.00,306.135199,1.644529,1.585032,946.000,786.000,379.661955,1.249676,0.398131,...,121.5,2.872281,0.0,-1.224242,1.033620,1.052048,0.093800,-3.285789e-01,-0.586684,1
augur,1000.20,884.00,340.076991,0.390810,-0.767813,1272.600,1173.500,433.187765,0.097656,-1.168206,...,707.5,2.872281,0.0,-1.224242,1.270574,1.286415,0.091287,-7.273991e-02,-1.082447,1


----------------------------------------
Top-level cluster: 1
shape: (49, 35)
label 0


...

----------------------------------------
label 2


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
pivx,5209.5,5309.0,346.315535,-0.502159,-0.641238,12.348,12.350,0.712205,0.501532,0.176144,...,0.276739,0.027618,-0.221106,-1.375208,4.442219,4.424418,0.544022,0.395436,-0.985103,2
decred,14698.8,14884.5,881.443566,-0.238279,-1.301848,112.917,114.000,5.571881,-0.418667,-0.179251,...,0.251751,0.021109,0.277673,-0.302655,6.777396,6.424316,1.530553,1.976658,3.106406,2
gas,15320.2,13127.0,4781.085186,1.331724,0.424041,60.042,54.745,12.639238,1.126204,-0.217662,...,1.792347,0.092517,-2.513322,4.638254,42.035974,0.021478,124.744681,2.666249,5.109803,2


----------------------------------------
label 3


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
digibyte,45244.0,47367.0,9629.632475,-0.168018,-0.382112,0.106681,0.1063,0.013395,0.002940,-1.026063,...,0.657323,0.076452,0.031175,-1.064487,179.278152,186.643661,58.957097,-0.895873,-0.395406,3
waves,9362.8,7282.0,7549.908037,1.994056,2.938267,12.091000,12.2850,1.010618,-0.062939,-1.529091,...,1.374007,1.875267,2.159137,3.338160,20.229100,17.963323,16.538838,0.388503,-1.396283,3


----------------------------------------
Top-level cluster: 3
shape: (1, 10)
Unclustered coins:
decentraland
bitshares
dentacoin
ripple
cardano
monacoin
zcoin
stratis
reddcoin
tether
tron
factom
bitcoin-diamond
maidsafecoin
zcash
omisego
eos
bitcoin
nem
verge
bytecoin-bcn
litecoin
bitcoin-cash
qtum
nuls
ethereum
dogecoin
bitsend
ark


## Clustering in bullish periods
    This result is more interesting and representative than previous one since number of bullish periods were discovered by algorithm. 

In [8]:
print_clusters(cl_coin_features_bullish)

Top-level cluster: 0
shape: (27, 80)
label 0


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,trans_per_address_median,trans_per_address_stddev,trans_per_address_skewns,trans_per_address_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
omisego,2864.672131,3214.000000,1552.299941,-0.014021,-0.902826,7.705902,7.95,3.706972,-0.059163,-0.221209,...,6689.247101,6458.275657,2.256706,7.049289,1.504531,1.298378,0.376701,1.049678,-0.372807,0
icon,1377.830065,1013.152941,796.776599,0.652353,-0.968316,1.670560,1.58,0.656504,0.751021,-0.421784,...,12099.545793,8415.579419,2.291495,5.057117,1.003832,0.826431,0.290387,0.866372,-0.949245,0


----------------------------------------
label 1


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,trans_per_address_median,trans_per_address_stddev,trans_per_address_skewns,trans_per_address_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
binance-coin,222.688525,155.0,308.682900,3.643297,15.090426,4.064302,1.750000,4.86378,1.095420,-0.644021,...,3275.618310,32087.845849,4.370334,19.332498,1.010247,1.00000,0.190931,0.459359,3.050089,1
vechain,511.577778,469.0,341.601944,2.314023,6.167152,1.226387,0.444544,1.29643,0.817719,-1.138559,...,8179.373849,6638.960188,1.494017,2.933845,1.002613,1.01432,0.081345,-0.496529,-0.223826,1


----------------------------------------
label 2


...

----------------------------------------
label 3


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
pivx,3868.592105,3493.0,1199.388870,0.498941,-0.614908,3.041447,3.045,1.225946,0.166806,-1.241401,...,0.181159,0.069845,0.006378,-0.561680,7.573134,4.689378,10.739344,4.866977,30.982242,3
decred,12609.973684,12851.0,2396.066749,0.025309,0.903741,38.111053,34.230,16.018983,0.033927,-1.357338,...,0.294930,0.058297,1.020762,0.382721,25.823752,6.876628,60.794957,4.177663,18.418250,3
gas,6007.039474,5991.5,8279.333479,4.016673,19.737099,16.308197,19.010,7.075478,-1.042342,-0.218823,...,1.759701,0.497833,0.893149,-0.105829,46.975880,0.045881,146.214560,3.214135,9.588394,3


----------------------------------------
label 4


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
dogecoin,51752.916031,35943.0,51612.542751,2.971437,8.162076,0.001317,0.000746,0.001224,1.307953,1.622682,...,0.463450,0.106373,0.988668,1.104292,23832.571361,14845.324429,17746.443909,1.762822,3.132042,4
digibyte,7684.783333,8482.0,4757.662685,-0.033255,-1.404691,0.008530,0.008040,0.009205,0.801146,-0.310219,...,0.662175,0.068527,-0.435285,-0.760036,372.447584,179.650167,534.834373,2.584212,7.528863,4


----------------------------------------
Top-level cluster: 3
shape: (2, 10)
Unclustered coins:
decentraland
zilliqa
bitshares
ripple
cybermiles
cryptonex
qash
cardano
monacoin
exclusivecoin
theta-token
mithril
transfercoin
stratis
aion
nebulas-token
power-ledger
tether
tron
zencash
factom
aelf
bytom
bitcoin-private
emercoin
hshare
maidsafecoin
tezos
eos
huobi-token
bitcoin
bitcoin-gold
golem-network-tokens
populous
verge
bytecoin-bcn
monetaryunit
wanchain
gifto
qtum
monaco
waves
iota


## Clustering in stable periods
Probably this part brings one of the most interesting results of whole research. Here are main reasons **why**:
1. A lot of stable periods was detected on BTC timeline. This makes this part the most representative.
2. In contrast to bullish and bearish BTC states this state can be postulated intuitively. Terms bullish and bearish are blurred and there no strong definitions of them, but everyone can say BTC is "in flat" now. In this context following results have the most substance to be used for predictive analytics applications.

In [9]:
print_clusters(cl_coin_features_stable)

Top-level cluster: 0
shape: (27, 80)
label 0


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,trans_per_address_median,trans_per_address_stddev,trans_per_address_skewns,trans_per_address_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
omisego,2672.818182,1849.5,3400.154002,4.257082,16.444122,14.033636,15.295,3.285743,-0.029376,-1.785547,...,8574.267110,7835.922589,3.403904,11.854217,1.250603,1.220757,0.092254,1.437914,1.315678,0
icon,1064.181818,1016.0,459.387798,1.230981,0.989221,3.469545,3.460,0.836184,0.139438,-1.556105,...,15882.261655,9311.066194,1.352512,1.869466,1.365978,1.339448,0.093221,0.370836,-0.958862,0
binance-coin,341.136364,268.0,448.434681,4.115746,15.686068,13.638182,13.505,0.961999,0.107986,-1.162500,...,5356.068593,6648.703983,1.364958,1.447460,1.101323,1.107300,0.072519,-0.276993,0.014251,0
vechain,754.545455,733.5,198.863984,0.926382,0.193645,3.863636,3.770,0.476212,0.739441,-0.458209,...,13111.603153,12632.468938,2.083868,3.440649,1.080126,1.081665,0.085020,0.377575,-0.853869,0


----------------------------------------
label 1


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,trans_per_address_median,trans_per_address_stddev,trans_per_address_skewns,trans_per_address_kurtos,tx_per_address_mean,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,label
zilliqa,744.545455,687.5,167.266724,0.823336,-0.047818,0.107056,0.108242,0.011798,-0.171165,-1.036350,...,21538.933731,8992.851056,1.089657,0.942628,1.553303,1.559909,0.196879,1.075706,1.157448,1
bytom,967.454545,692.0,842.151559,1.490960,1.400607,0.833762,0.895669,0.194366,0.061376,-1.633310,...,11989.210548,66598.409677,4.331024,16.862780,1.048715,1.007277,0.114336,0.755579,-0.377343,1
aeternity,786.318182,733.0,522.398349,1.009808,-0.125849,2.518597,2.760000,1.307090,-0.000262,-0.883880,...,15340.888632,13701.400943,0.723772,-0.780341,1.445409,1.357535,0.285551,0.512212,-1.107613,1
populous,289.681818,296.5,59.350566,-0.104536,-1.317464,19.224545,21.400000,5.901751,-0.040507,-1.769804,...,9889.868868,7686.612149,1.632045,2.006220,1.241779,1.156602,0.273182,1.244534,0.371802,1


----------------------------------------
label 2


...

----------------------------------------
label 2


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
pivx,1846.432836,1610.0,1110.457994,2.059233,5.587268,0.471289,0.004237,1.278461,3.002248,8.255485,...,0.181687,0.211016,0.912007,0.005098,109.293277,0.079228,1837.191326,17.116920,290.993789,2
dogecoin,37031.095436,29659.0,25521.400260,3.500841,15.641470,0.000400,0.000221,0.000818,4.835900,23.915396,...,0.449964,0.139657,0.924589,0.953875,49570.341686,26793.806596,504027.586361,26.603118,707.466927,2
digibyte,5312.140083,3570.0,4583.647352,1.457666,1.704523,0.001649,0.000216,0.006809,4.889737,23.228667,...,0.601394,0.166856,0.428372,1.704266,2082.153333,136.889135,3096.069237,2.689101,11.455351,2


----------------------------------------
label 3


,active_address_mean,active_address_median,active_address_stddev,active_address_skewns,active_address_kurtos,close_mean,close_median,close_stddev,close_skewns,close_kurtos,...,tx_per_address_median,tx_per_address_stddev,tx_per_address_skewns,tx_per_address_kurtos,mdiff_to_volatility_mean,mdiff_to_volatility_median,mdiff_to_volatility_stddev,mdiff_to_volatility_skewns,mdiff_to_volatility_kurtos,label
decred,9383.055215,8648.0,4416.979858,1.177938,2.018685,8.831842,1.610000,21.839712,3.181559,8.866928,...,0.322929,0.077495,0.669991,3.478893,89.648337,32.720657,159.313870,3.346004,12.079611,3
verge,1677.131012,485.0,5652.613896,5.455874,29.760513,0.002192,0.000021,0.011193,5.472196,29.579790,...,0.747899,0.147967,-0.817485,1.265788,579.283127,16.435895,1860.248746,6.385463,47.934350,3
waves,2706.739583,834.5,4270.021684,2.404568,6.808030,0.879876,0.228037,1.644724,2.362272,4.208934,...,4.173801,5.306126,1.814269,2.780518,49.596648,2.959382,173.604198,5.248254,27.135744,3


----------------------------------------
Top-level cluster: 3
shape: (3, 10)
Unclustered coins:
moac
decentraland
ripple
polymath-network
pundi-x
cryptonex
cardano
monacoin
siacoin
elastos
theta-token
mithril
reddcoin
stratis
ethereum-classic
lisk
0x
nebulas-token
ontology
gas
tether
tron
factom
bitcoin-private
wax
bitcoin-diamond
bitcoin
eos
zcash
gxchain
augur
bitcoin-gold
golem-network-tokens
gifto
wanchain
bitcoin-cash
qtum
monaco
nuls
ethereum
bitsend
ark


# Further steps
Final part of research was successfully finished. There are a lot of potential improvements can be made to polish results obtained. This goes a bit beyond of currend report, since these results are interesting itself and should be additionally studied. So extra part IV of report will be delivered to present and summarize results of coins clustering with implemented methods.